In [1]:


# Initialize the model
model = "llama3:8b-instruct-q4_K_M"



In [2]:
import chromadb

# Initialize Chroma DB client
client = chromadb.PersistentClient(path='database')

In [11]:
my_collection = client.create_collection(
        name="collection1",
        metadata={"hnsw:space": "cosine"} # l2 is the default
    )

In [10]:

client.delete_collection(name="collection1")

In [7]:
pip install pypdf

Note: you may need to restart the kernel to use updated packages.


In [4]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("book.pdf")
pages = loader.load_and_split()

In [5]:
print(pages)

[Document(metadata={'source': 'book.pdf', 'page': 0}, page_content='Courtesy of www.pdfbooksfree.pk'), Document(metadata={'source': 'book.pdf', 'page': 1}, page_content='1 COMMENT: \nAll Praise is due to Allaah, our Creator, Nourisher and Provider. Peace and \nBlessings be upon all the Ambiyaa (A.S.) and upon the last and final Rasul (Messenger) - Muhammad (S.A.W.). Peace and blessings upon his \ncompanions who accepted and propagated DEEN (religion) to the entire \nworld. \nThe book “Principles of Deen” has 12 chapters, explaining: \n1.) Wahy ; \n2.) Principles of Tafseer ; \n3.) Principles of Hadith ; \n4.) Consensus of the Ummat (Ijma) ; \n5.) Qiyaas ; \n6.) What is Fiqh ; \n7.) Principles of Fiqh ; \n8.) Essentials of Fiqh ; \n9.) Principles of Ijtihaad ; \n10.) Principles of Taqleed ; \n11.) Principles of Sunant and Bidaat and \n12.) Principles of Imaan and Kufr. \nInformation in this book, because of its close link to classically written text \nwill be new to many readers. Thus I

In [6]:
all_text = "\n".join(page.page_content for page in pages)

print(all_text)

Courtesy of www.pdfbooksfree.pk
1 COMMENT: 
All Praise is due to Allaah, our Creator, Nourisher and Provider. Peace and 
Blessings be upon all the Ambiyaa (A.S.) and upon the last and final Rasul (Messenger) - Muhammad (S.A.W.). Peace and blessings upon his 
companions who accepted and propagated DEEN (religion) to the entire 
world. 
The book “Principles of Deen” has 12 chapters, explaining: 
1.) Wahy ; 
2.) Principles of Tafseer ; 
3.) Principles of Hadith ; 
4.) Consensus of the Ummat (Ijma) ; 
5.) Qiyaas ; 
6.) What is Fiqh ; 
7.) Principles of Fiqh ; 
8.) Essentials of Fiqh ; 
9.) Principles of Ijtihaad ; 
10.) Principles of Taqleed ; 
11.) Principles of Sunant and Bidaat and 
12.) Principles of Imaan and Kufr. 
Information in this book, because of its close link to classically written text 
will be new to many readers. Thus I suggest one reads, studies and seeks further explanation from 
Ulema. In fact, it is an ideal text book for advanced adult classes in any part 
of the world

Cleaning extracted text 

In [7]:
import re

# Sample text data loaded from PDF


# Define the unwanted text pattern
unwanted_text_pattern = r'Courtesy of www\.pdfbooksfree\.pk'

unwanted_text_pattern1 = r'1 COMMENT:'
# Remove the unwanted text using regular expressions
cleaned_text = re.sub(unwanted_text_pattern, '', all_text)
# cleaned_text = re.sub(unwanted_text_pattern1, '', all_text)
# Remove extra newlines and whitespace
cleaned_text = re.sub(r'\n\s*\n', '\n', cleaned_text).strip()

print(cleaned_text)


1 COMMENT: 
All Praise is due to Allaah, our Creator, Nourisher and Provider. Peace and 
Blessings be upon all the Ambiyaa (A.S.) and upon the last and final Rasul (Messenger) - Muhammad (S.A.W.). Peace and blessings upon his 
companions who accepted and propagated DEEN (religion) to the entire 
world. 
The book “Principles of Deen” has 12 chapters, explaining: 
1.) Wahy ; 
2.) Principles of Tafseer ; 
3.) Principles of Hadith ; 
4.) Consensus of the Ummat (Ijma) ; 
5.) Qiyaas ; 
6.) What is Fiqh ; 
7.) Principles of Fiqh ; 
8.) Essentials of Fiqh ; 
9.) Principles of Ijtihaad ; 
10.) Principles of Taqleed ; 
11.) Principles of Sunant and Bidaat and 
12.) Principles of Imaan and Kufr. 
Information in this book, because of its close link to classically written text 
will be new to many readers. Thus I suggest one reads, studies and seeks further explanation from 
Ulema. In fact, it is an ideal text book for advanced adult classes in any part 
of the world. 
The contents, if understood, 

CHUNKER BASE CLASS

In [12]:
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
class ChunkerBase:
    def __init__(self, model_name="all-MiniLM-L6-v2", max_chunk_size=800):
        self.embeddings_model = HuggingFaceEmbeddings(model_name=model_name)
        self.chunks = []
        self.ids=[]
        self.chunk_embeddings = []
        self.collection = my_collection  # Use ChromaDB collection for storing documents
        self.max_chunk_size = max_chunk_size

    def embed_chunks(self):
        print("Num of input chunks: ", len(self.chunks))
        self.chunk_embeddings = self.embeddings_model.embed_documents(self.chunks)
        print("Num of retrieved embeddings for chunks: ", len(self.chunk_embeddings))
        documents=[]
        # list(map(str, self.chunks))
        # embeddings_as_lists = [embedding for embedding in self.chunk_embeddings]


        self.ids = [f"idx{i+1}" for i in range(len(self.chunks))]

        print(self.chunks)
        print(self.ids)
        # self.collection.add(self.chunks,self.ids)
        self.collection.add(embeddings=self.chunk_embeddings, documents=self.chunks, ids=self.ids)

    

    def embed_query(self, query):
        return self.embeddings_model.embed_documents([query])[0]

    def search_similar_chunks(self, query_embedding, top_k=7):
        print(query_embedding)
        results = self.collection.query(query_embedding, n_results=top_k)

        if not results:
            print("No similar chunks found.")
            return [], []

        print(results)

        similar_chunks = []
        for doc_list, score_list in zip(results['documents'], results['distances']):
            for doc, score in zip(doc_list, score_list):
                similar_chunks.append((doc, score))

        return similar_chunks


    def get_all_chunks(self):
        return self.chunks

RECURSIVE CHUNKER CLASS 

In [13]:
class RecursiveChunker(ChunkerBase):
    def __init__(self, model_name="all-MiniLM-L6-v2", max_chunk_size=750, max_paragraph_words=750):
        super().__init__(model_name, max_chunk_size)
        self.max_paragraph_words = max_paragraph_words

    def _split_sentences(self, text):
        return re.split(r'(?<=[.?!])\s+', text)

    def recursive_chunk_text(self, text):
        words = text.split()

        if len(words) <= self.max_chunk_size:
            return [text]

        # Find the best splitting point
        split_point = self.max_chunk_size
        while split_point > 0 and not words[split_point - 1].endswith(('.', '!', '?')):
            split_point -= 1

        if split_point == 0:  # No good splitting point found, force split at max_chunk_size
            split_point = self.max_chunk_size

        # Split the text
        chunk = ' '.join(words[:split_point])
        remaining_text = ' '.join(words[split_point:])

        # Recursive call
        chunks = [chunk] + self.recursive_chunk_text(remaining_text)

        return chunks

    def chunk_text(self, text):
        paragraphs = text.split('\n\n\n')
        self.chunks = []

        for paragraph in paragraphs:
            if len(paragraph.split()) > self.max_paragraph_words:
                indented_chunks = self.recursive_chunk_text(paragraph)
                self.chunks.extend(indented_chunks)
            else:
                self.chunks.append(paragraph)

        print('Total number of chunks:', len(self.chunks))
    def save_chunks_to_file(self, filename):
        with open(filename, 'w') as f:
            for chunk in self.chunks:
                f.write(chunk + '\n\n')

    def read_text_from_file(self, file_path):
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()
        return text

# Initialize the chunker
chunker = RecursiveChunker()

text_of_book=cleaned_text
chunker.chunk_text(text_of_book)
chunker.embed_chunks()
chunker.save_chunks_to_file("chunks.txt")
print('Total number of chunks:', len(chunker.chunks))




c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
c:\Users\MY IT STORE\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be re

Total number of chunks: 73
Num of input chunks:  73
Num of retrieved embeddings for chunks:  73
['1 COMMENT: All Praise is due to Allaah, our Creator, Nourisher and Provider. Peace and Blessings be upon all the Ambiyaa (A.S.) and upon the last and final Rasul (Messenger) - Muhammad (S.A.W.). Peace and blessings upon his companions who accepted and propagated DEEN (religion) to the entire world. The book “Principles of Deen” has 12 chapters, explaining: 1.) Wahy ; 2.) Principles of Tafseer ; 3.) Principles of Hadith ; 4.) Consensus of the Ummat (Ijma) ; 5.) Qiyaas ; 6.) What is Fiqh ; 7.) Principles of Fiqh ; 8.) Essentials of Fiqh ; 9.) Principles of Ijtihaad ; 10.) Principles of Taqleed ; 11.) Principles of Sunant and Bidaat and 12.) Principles of Imaan and Kufr. Information in this book, because of its close link to classically written text will be new to many readers. Thus I suggest one reads, studies and seeks further explanation from Ulema. In fact, it is an ideal text book for ad

PRINTING SIMILAR CHUNKS TO QUERY

In [14]:
# Query similar chunks
query = "What is the difference between Hadith and Sunnat?"
query_embedding = chunker.embed_query(query)

# Search for similar chunks
similar_chunks = chunker.search_similar_chunks(query_embedding,3 )

print(f"Top {len(similar_chunks)} Similar Chunks for Query: '{query}'")
print("=" * 50)
for i, (chunk_content, similarity_score) in enumerate(similar_chunks, start=1):
    print(f"{i}. Similarity Score: {similarity_score:.4f}")
    print(f"   Chunk Content: {chunk_content}")
    print("-" * 50)




[0.017977271229028702, 0.1107054129242897, -0.04532354325056076, 0.051356974989175797, 0.015625834465026855, -0.01135567482560873, 0.009815467521548271, -0.012659450992941856, 0.0515005849301815, -0.009384837932884693, -0.0202617384493351, -0.04591984674334526, 0.023723140358924866, -0.01431987900286913, 0.033803123980760574, -0.009648925624787807, -0.006776513531804085, 0.01764587312936783, 0.025618204846978188, -0.01409874577075243, 0.021748526021838188, 0.02561992034316063, -0.01384412869811058, -0.0597277507185936, 0.040941040962934494, 0.005700183566659689, -0.07276102900505066, -0.012735940515995026, 0.0023467023856937885, -0.0672665685415268, -0.011057620868086815, 0.03854956105351448, -0.01656409353017807, 0.027173593640327454, -0.11153903603553772, 0.02496454492211342, 0.00984302069991827, 0.03144402801990509, -0.020939016714692116, -0.05115644261240959, 0.0008128555491566658, -0.036919962614774704, -0.06907892227172852, 0.014738134108483791, -0.03849537670612335, 0.0706774592

In [14]:


reordered_chunks = [similar_chunks[2], similar_chunks[0], similar_chunks[1]]

# Join the reordered chunks into the result_string
result_string = " ".join([text for text, score in reordered_chunks])

In [16]:
print(result_string)

Similarly, those matters which are special and exclusive to Nabi (sallallahu alaihi wasallam), are also not included as part of the Sunnat, since they cannot be followed by others, like marrying more than four wives, making Nikah without Mehr, etc. However, they are part of the Hadith. Hadith The term Hadith (which literally means ‘new’) is actually the antonym of ‘old’ ( qadeem). Its connotation is with speech. This original meaning of the word ‘Hadith’ is kept in the fore when defining it according to the Shariah. The Qur`aan Majeed is the Speech of Allaah Ta`ala, which is part of His Qadeem (ever-existent) Qualities. Hence the speech, actions and discourses of Nabi (sallallahu alaihi wasallam) are ‘Hadith’. According to the Shariah, the term ‘Hadith’ has two connotations: 38 The first connotation of ‘Hadith’ The account of Nabi (sallallahu alaihi wasallam), his speech, actions and discourses, are known as Hadith. In other words, this is known as the text of Hadith. The difference be

In [ ]:
# Query similar chunks
query1 = "What is the difference?"
query_embedding = chunker.embed_query(query1)

# Search for similar chunks
similar_chunks = chunker.search_similar_chunks(query_embedding,3 )

print(f"Top {len(similar_chunks)} Similar Chunks for Query: '{query}'")
print("=" * 50)
for i, (chunk_content, similarity_score) in enumerate(similar_chunks, start=1):
    print(f"{i}. Similarity Score: {similarity_score:.4f}")
    print(f"   Chunk Content: {chunk_content}")
    print("-" * 50)

result_string1 = " ".join([text for text, score in similar_chunks])


In [22]:
print(result_string)

Based on this, Sunnat is best described as “A path which is followed” . This could refer to both, a good and evil path. Those who have stated it as being a good action, merely state so by way of example and illustration. A Sunnat is not necessarily attributed to only a good act. In the Qur`aan Majeed and Ahaadith it is used in both contexts. As far as attributing it to a good act, it is stated in the Qur`aan Majeed, “The Sunnat of those prophets We had sent before you.” [Surah Israa] In context of an evil act, it is stated in the Qur`aan Majeed, “In this way We place in the hearts of the evildoers, they do not bring Imaan in the Prophet. Indeed the way (Sunnat) of the previous nations has been laid out.” [Surah Hajar] Similarly, the word Sunnat has been used in both contexts in one Hadith, “Whosoever initiates a good Sunnat, for him is its reward and the reward of whoever else follows it, without their reward being diminished in the least bit. And whoever initiates an evil Sunnat, for 

USING SYSTEM USER SYNTAX TO SEND PROMPT TO LLM

In [21]:
import ollama
query3="What is the difference between Hadith and Sunnat?"
response2 = ollama.chat(model='llama3:8b-instruct-q4_K_M', messages=[
    {
        'role': 'system',
        'content': f"Using the document provided below, strictly provide the answer to the question. If you can not find an answer in the document, respond with 'I am sorry. I don't know the answer.' \n\n***{result_string}***\n\n"
    },
    {
        'role': 'user',
        'content': f"Question :{query3}"
    }
])

print(response2['message']['content'])

According to the document:

"The first connotation of ‘Hadith’

...the account of Nabi (sallallahu alaihi wasallam), his speech, actions and discourses, are known as Hadith. In other words, this is known as the text of Hadith.

The difference between ‘Hadith’ and ‘Sunnat’, with regard to this first connotation: Following and imitation is not included in the definition of ‘Hadith’, whereas this is part of the meaning of ‘Sunnat’.

Hence, whereas the literal definition of ‘Hadith’ and ‘Sunnat’ may be the same, the difference lies in the practical application of the two. Those statements, practises and discourses of Nabi (sallallahu alaihi wasallam) which are possible to follow are regarded as Sunnat, and these are also included as part of the Hadith, together with those statements, practises and discourses which are not possible of following.

In essence, ‘Hadith’ is general, which includes Sunnat and non-Sunnat, whereas Sunnat is specific. It is as though, every Sunnat is part of the Ha

FOR SETTING PARAMETERS FOR RESPONSE FROM MODEL

In [24]:
import ollama
query3="What is the difference between Hadith and Sunnat?"
response3 = ollama.chat(
    model='llama3:8b-instruct-q4_K_M',
    messages=[
        {
            'role': 'system',
            'content': f"Using the document provided below, strictly provide the answer to the question. If you can not find an answer in the document, respond with 'I am sorry. I don't know the answer.' \n\n***{result_string}***\n\n"
        },
        {
            'role': 'user',
            'content': f"Question :{query3}"
        }
    ],
    options={
        'temperature': 0.7,
        'top_k': 40,
        # Add other parameters as needed
    }
)
print(response3['message']['content'])

According to the document:

The first connotation of 'Hadith' refers to the account of Nabi (sallallahu alaihi wasallam), his speech, actions, and discourses. This includes both statements, practices, and discourses that are possible to follow as well as those that are not.

On the other hand, 'Sunnat' refers specifically to those statements, practices, and discourses of Nabi (sallallahu alaihi wasallam) that are possible to follow. In essence, 'Hadith' is general and includes both Sunnat and non-Sunnat, whereas Sunnat is specific.

In other words, every Sunnat is part of the Hadith, but not every Hadith is part of the Sunnat.


In [15]:
import ollama
query3="What is the difference between Hadith and Sunnat?"
response13 = ollama.chat(
    model='llama3:8b-instruct-q4_K_M',
    messages=[
        {
            'role': 'system',
            'content': f"Using the document provided below, strictly provide the answer to the question. If you can not find an answer in the document, respond with 'I am sorry. I don't know the answer.' \n\n***{result_string}***\n\n"
        },
        {
            'role': 'user',
            'content': f"Question :{query3}"
        }
    ],
    options={
        'temperature': 0.7,
        'top_k': 40,
        # Add other parameters as needed
    }
)
print(response13['message']['content'])

According to the provided document:

Hadith refers to the narration of the Prophet's (sallallahu alaihi wasallam) speech and actions, whereas Sunnah refers to his actions.

In other words, every action of Nabi (sallallahu alaihi wasallam) is regarded as a Sunnah, and its narration is referred to as Hadith.


USING 4096 CONTEXT WINDOW SIZE

In [17]:
import ollama
query3="What is the difference between Hadith and Sunnat?"
response14 = ollama.chat(
    model='llama3:8b-instruct-q4_K_M',
    messages=[
        {
            'role': 'system',
            'content': f"Using the document provided below, strictly provide the answer to the question. If you can not find an answer in the document, respond with 'I am sorry. I don't know the answer.' \n\n***{result_string}***\n\n"
        },
        {
            'role': 'user',
            'content': f"Question :{query3}"
        }
    ],
    options={
        'temperature': 0.7,
        'top_k': 40,
        'num_ctx': 4096,
        # Add other parameters as needed
    }
)
print(response14['message']['content'])

According to the document provided, the difference between Hadith and Sunnat lies in their connotations:

1. The first connotation of Hadith refers to the account of Nabi (sallallahu alaihi wasallam), his speech, actions, and discourses, which are known as Hadith.
2. Following and imitation is not included in the definition of Hadith, whereas it is part of the meaning of Sunnat.

On the other hand, Sunnat refers to "A path which is followed" and includes both good and evil paths. In the context of Deen (Islamic faith), Sunnat means following the path of Nabi (sallallahu alaihi wasallam) and the Sahaabah (radhiallahu anhum).

The document also states that Hadith, according to its second connotation, refers to the combination of text and chain of narrators. This connotation includes the classification of strong and weak Ahaadith.

In summary, Hadith refers to the account of Nabi's speech, actions, and discourses, whereas Sunnat refers to "A path which is followed" in Deen, including foll

FOR GENERATING RESPONSE WORD BY WORD

In [23]:
from ollama import chat
messages = [
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
]
model = "llama3:8b-instruct-q4_K_M"

for part in chat(model=model, messages=messages, stream=True):
  print(part['message']['content'], end='', flush=True)

# end with a newline
print()

The sky appears blue because of a phenomenon called scattering, which occurs when sunlight interacts with tiny molecules of gases in the Earth's atmosphere. Here's a simplified explanation:

1. **Sunlight enters the Earth's atmosphere**: When the sun emits light, it consists of a wide range of colors, including all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Light is scattered by atmospheric molecules**: As sunlight travels through the atmosphere, it encounters tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
3. **Short wavelengths (blue and violet) scatter more**: The smaller molecules scatter shorter wavelengths of light, such as blue and violet, more efficiently than longer wavelengths, like red and orange. This is known as Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described the phenomenon in the late 19th century.
4. **B

USING 1024 CONTEXT WINDOW SIZE 

In [25]:
response3 = ollama.chat(
    model='llama3:8b-instruct-q4_K_M',
    messages=[
        {
            'role': 'system',
            'content': f"Using the document provided below, strictly provide the answer to the question. If you can not find an answer in the document, respond with 'I am sorry. I don't know the answer.' \n\n***{result_string}***\n\n"
        },
        {
            'role': 'user',
            'content': f"Question :{query3}"
        }
    ],
    options={
        'num_ctx': 1024,  # Set context length
        'temperature': 0.7,
        'top_k': 40,
        'top_p': 0.9,
        'repeat_penalty': 1.1,
    },
    stream=True
)

# To handle the streaming response:
for chunk in response3:
    print(chunk['message']['content'], end='', flush=True)

According to the document, the difference lies in their connotations:

* "Hadith" (according to its second connotation) refers to the verified Ahaadith that have been compiled under certain principles and regulations, which discern the acceptable Ahaadith from the unacceptable.
* "Sunnat", on the other hand, is not properly defined or categorized in the same way as Hadith.